In [34]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# ---------------------- INICIALIZACIÓN ----------------------

# Intenta leer el archivo CSV para obtener las URLs ya procesadas.
# Si no existe el archivo, crea uno nuevo.
try:
    df_peliculas = pd.read_csv("data/data_peliculas.csv")
    urls_procesadas = df_peliculas.loc[df_peliculas['Proceso'] == 'ok', 'URL'].tolist()
except FileNotFoundError:
    urls_procesadas = []
    # Crear el archivo CSV con encabezados si no existe
    with open("data/data_peliculas.csv", "w") as file:
        file.write("Nombre,URL,Proceso\n")

url = "https://www.filmaffinity.com/es/topgen.php?genres=&chv=0&orderby=avg&movietype=movie%7C&country=ES&fromyear=1900&toyear=2023&ratingcount=1&runtimemin=0&runtimemax=4"

# ---------------------- NAVEGADOR ----------------------

# Crea una instancia del navegador
driver = webdriver.Chrome()
# Carga la página web
driver.get(url)

# Espera a que la página se cargue
time.sleep(5)

# Aceptar las cookies
try:
    cookie_button = driver.find_element(By.CLASS_NAME, 'css-v43ltw')
    cookie_button.click()
except Exception as e:
    print(f"No se encontró el botón de cookies: {e}")

def click_pagina():
    # Espera a que el botón "Ver más resultados" esté presente en la página
    wait = WebDriverWait(driver, 10)
    chevron_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.z-top-movies i.fas.fa-chevron-down')))
    driver.execute_script("arguments[0].click();", chevron_button)

    # Espera a que la página se actualice después de hacer clic en el botón
    time.sleep(5)  # Ajusta el tiempo según sea necesario

def extract_movies_data(urls_procesadas):
    # Encuentra la sección "top-movies"
    soup = BeautifulSoup(driver.page_source, "html.parser")
    movies = soup.find("main", id="mt-content-cell").find_all("div", class_="mc-title")

    # Inicializa listas para almacenar los títulos y URLs
    titles = []
    urls = []

    # Extrae los títulos y URLs
    for movie in movies:
        title = movie.a.get('title').strip()
        url = movie.a.get('href').strip()
        if url not in urls_procesadas:
            titles.append(title)
            urls.append(url)

    # Crea un DataFrame
    df = pd.DataFrame({'Title': titles, 'URL': urls, 'Procesado': 'no'})
    return df

# Inicializa un DataFrame vacío para almacenar la información
df_peliculas = pd.DataFrame(columns=['Title', 'URL', 'Procesado'])

# Bucle para alternar entre las funciones y guardar la información
while True:
    click_pagina()
    df = extract_movies_data(urls_procesadas)
    if not df.empty:
        df_peliculas = pd.concat([df_peliculas, df])
        df_peliculas.to_csv("data/data_peliculas.csv", index=False)
        urls_procesadas.extend(df['URL'].tolist())
        print(f"Descargando información de {len(df)} películas...")
    else:
        print("No hay más películas para procesar.")
        break

# Cierra el navegador
driver.quit()

# Muestra el DataFrame
print(f"Total de películas descargadas: {len(df_peliculas)}")
df_peliculas

Descargando información de 60 películas...
Descargando información de 30 películas...
Descargando información de 30 películas...
Descargando información de 30 películas...
Descargando información de 30 películas...
Descargando información de 30 películas...
Descargando información de 30 películas...
Descargando información de 30 películas...
Descargando información de 30 películas...
Descargando información de 30 películas...
Descargando información de 30 películas...
Descargando información de 30 películas...
Descargando información de 30 películas...
Descargando información de 30 películas...
Descargando información de 30 películas...
Descargando información de 30 películas...
Descargando información de 30 películas...
Descargando información de 30 películas...
Descargando información de 30 películas...
Descargando información de 30 películas...
Descargando información de 30 películas...
Descargando información de 30 películas...
Descargando información de 30 películas...
Descargando

TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x0000000106e5be08 chromedriver + 5025288
1   chromedriver                        0x0000000106e52c23 chromedriver + 4987939
2   chromedriver                        0x00000001069f4e67 chromedriver + 409191
3   chromedriver                        0x0000000106a441b9 chromedriver + 733625
4   chromedriver                        0x0000000106a44371 chromedriver + 734065
5   chromedriver                        0x0000000106a8a194 chromedriver + 1020308
6   chromedriver                        0x0000000106a6b50d chromedriver + 894221
7   chromedriver                        0x0000000106a87571 chromedriver + 1009009
8   chromedriver                        0x0000000106a6b2b3 chromedriver + 893619
9   chromedriver                        0x0000000106a35eb9 chromedriver + 675513
10  chromedriver                        0x0000000106a370ee chromedriver + 680174
11  chromedriver                        0x0000000106e1d819 chrome

In [9]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm
import datetime
import random
import time

# 1. DEFINICIÓN DE CLASES Y FUNCIONES ----------------------

class Golem:
    def __init__(self, navegador):
        self.navegador = navegador

    def generar_url_aleatoria_wikipedia(self):
        return 'https://es.wikipedia.org/wiki/Especial:Aleatoria'

    def accion(self, url_destino):
        accion_elegida = random.choice(['esperar', 'navegar'])
        if accion_elegida == 'esperar':
            tiempo_espera = random.uniform(3, 4)
            time.sleep(tiempo_espera)
        else:
            url = self.generar_url_aleatoria_wikipedia()
            self.navegador.get(url)
            tiempo_espera = random.uniform(2, 5)
            time.sleep(tiempo_espera)
        self.navegador.get(url_destino)

def extraer_info_pelicula(url, navegador, golem):
    golem.accion(url)
    soup = BeautifulSoup(navegador.page_source, 'html.parser')
    info = {}
    
    #info['Titulo'] = soup.find('h1', {'id': 'main-title'}).text.strip() if soup.find('h1', {'id': 'main-title'}) else "No se encontró información del título"
    
    info["Nota"] = float(soup.find('div', {'id': 'movie-rat-avg'}).get("content")) if soup.find('div', {'id': 'movie-rat-avg'}) else "No se encontró información de nota"
    
    info["Votos"] = int(soup.find('span', {'itemprop': 'ratingCount'}).get("content")) if soup.find('span', {'itemprop': 'ratingCount'}) else "No se encontró información de votos"
    
    info['Año'] = int(soup.find('dd', {'itemprop': 'datePublished'}).text.split()[0]) if soup.find('dd', {'itemprop': 'datePublished'}) else "No se encontró información del año"
    
    actor_tags = soup.find_all('div', {'class': 'name'})
    actor_names = [tag.get_text() for tag in actor_tags]
    info["Reparto"] = ', '.join(actor_names) if actor_names else "No se encontró información de reparto"
    
    director_span = soup.find('span', {'class': 'nb', 'itemprop': 'director'})
    info['Director'] = director_span.find('span', {'itemprop': 'name'}).text if director_span and director_span.find('span', {'itemprop': 'name'}) else "No se encontró información del director"
    
    card_producer = soup.find("dd", class_="card-producer")
    if card_producer:
        info["Compañias"] = ', '.join([a.text for a in card_producer.select('a')])
    else:
        info["Compañias"] = "No se encontró información de compañías"
    
    producer_dd = soup.find('dd', class_='card-producer')
    coproduction_text = producer_dd.get_text(strip=True) if producer_dd else ""
    countries = [word for word in coproduction_text.split() if word != 'España']
    coproduction_country = countries[1] if len(countries) > 1 else "No se encontró información de coproducción"
    info["Coproduccion"] = coproduction_country
    
    generos_dd = soup.find("dd", class_="card-genres")
    info["Generos"] = ', '.join([a.text for a in generos_dd.select('a')]) if generos_dd and generos_dd.select('a') else "No se encontró información de géneros"
    
    award_dd = soup.find('dd', class_='award')
    award_divs = award_dd.find_all('div', class_='margin-bottom') if award_dd else []
    premios = [div.get_text() for div in award_divs]
    info["Premios"] = ', '.join(premios)
    
    # Convertir el diccionario en un DataFrame
    data = pd.DataFrame([info])

    return data

# 2. INICIALIZACIÓN DE DATOS ----------------------

try:
    df_peliculas = pd.read_csv("data_peliculas.csv", low_memory=False)
except FileNotFoundError:
    print("Error: No se encontró el archivo data_peliculas.csv'")
    exit()

navegador = webdriver.Chrome()
golem = Golem(navegador)

start_time = datetime.datetime.now()

# 3. PROCESO PRINCIPAL ----------------------

for idx, row in tqdm(df_peliculas.iterrows(), total=df_peliculas.shape[0], desc="Procesando películas"):
    if row['Procesado'] == "Sí":
        continue

    url_pelicula = row['URL']

    try:
        pelicula_df = extraer_info_pelicula(url_pelicula, navegador, golem)
        for col in pelicula_df.columns:
            df_peliculas.at[idx, col] = pelicula_df.at[0, col]

        df_peliculas.at[idx, "Procesado"] = "Sí"
    except Exception as e:
        print(f"Error al procesar la película con URL: {url_pelicula}. Detalle del error: {e}")
        navegador.quit()
        df_peliculas.to_csv("data/data_peliculas.csv", index=False)
        break

    if (idx + 1) % 10 == 0:
        df_peliculas.to_csv("data/data_peliculas.csv", index=False)

# 4. FINALIZACIÓN Y GUARDADO ----------------------      

df_peliculas.to_csv("data/data_peliculas.csv", index=False)

navegador.quit()

end_time = datetime.datetime.now()
total_time = end_time - start_time
print(f"Proceso finalizado en {total_time}.")

Procesando películas: 100%|██████████| 3288/3288 [00:00<00:00, 17838.54it/s]


Proceso finalizado en 0:00:00.402149.


In [11]:
# Mostrar el DataFrame final
df_peliculas

,Title,URL,Procesado,Nota,Votos,Año,Reparto,Director,Compañias,Coproduccion,Generos,Premios
0,El verdugo,https://www.filmaffinity.com/es/film411856.html,Sí,8.2,41447.0,1963.0,"José Isbert, Nino Manfredi, Emma Penella, José...",Luis García Berlanga,"Naga Films, Zabra Films",España-Italia;Naga,"Comedia, Sátira, Comedia negra, Película de culto",1963: Festival de Venecia: Premios FIPRESCI
1,Los santos inocentes,https://www.filmaffinity.com/es/film879812.html,Sí,8.1,44470.0,1984.0,"Alfredo Landa, Francisco Rabal, Juan Diego, Te...",Mario Camus,"Ganesh, RTVE",No se encontró información de coproducción,"Drama, Drama social, Vida rural, Años 60, Caza",1984: Cannes: Mejor Actor (ex aequo: Francisco...
2,Plácido,https://www.filmaffinity.com/es/film895672.html,Sí,8.1,20396.0,1961.0,"Cassen, José Luis López Vázquez, Elvira Quinti...",Luis García Berlanga,"Jet Films, Alfredo Matas",Films.Productor:Alfredo,"Comedia, Pobreza, Navidad, Sátira",1961: Nominada al Oscar: Mejor película de hab...
3,Viridiana,https://www.filmaffinity.com/es/film123112.html,Sí,8.1,28771.0,1961.0,"Silvia Pinal, Fernando Rey, Francisco Rabal, M...",Luis Buñuel,"Films 59, Uninci",España-México;Films,"Drama, Religión, Película de culto",1961: Festival de Cannes: Palma de Oro (ex-aequo)
4,"Bienvenido, Míster Marshall",https://www.filmaffinity.com/es/film435869.html,Sí,8.0,31421.0,1953.0,"José Isbert, Lolita Sevilla, Manolo Morán, Alb...",Luis García Berlanga,Uninci,No se encontró información de coproducción,"Comedia, Sátira",1953: Festival de Cannes: Premio Internacional...
...,...,...,...,...,...,...,...,...,...,...,...,...
3283,Mallorca Zombie,https://www.filmaffinity.com/es/film614286.html,Sí,1.6,236,2009,No se encontró información de reparto,Marc Albertí,"Alex Vargas, Friki Films","Vargas,Friki","Terror, Zombis, Serie B, Película de episodios",NaN
3284,El altar: Demasiado tarde para rezar,https://www.filmaffinity.com/es/film166981.html,Sí,1.6,123,2013,No se encontró información de reparto,Pedro Sánchez Román,Grafxdigital,No se encontró información de coproducción,"Terror, Fantástico",NaN
3285,Bocadillo,https://www.filmaffinity.com/es/film513377.html,Sí,1.5,2073,2018,"Ismael Prego, Joaquín Albero, David Cajal, Mar...",Ismael Prego,YouPlanet,No se encontró información de coproducción,Comedia,NaN
3286,2097,https://www.filmaffinity.com/es/film680362.html,Sí,1.5,102,2017,"Saray Castro, Abraham Santacruz, Alba Martín, ...",Esteban Calderín,No se encontró información de compañías,No se encontró información de coproducción,Acción,NaN


In [ ]:
HERRAMIENTAS DE LIMPIEZA*****************************

In [ ]:
*****************************************************

In [45]:
import pandas as pd

# Realiza un reset de la columna Procesado, para volver a procesar
# Leer el archivo CSV
file_path = 'data/data_peliculas.csv'
data = pd.read_csv("data/data_peliculas.csv", low_memory=False)

# Cambiar todos los valores en la columna 'Procesado' a 'No'
data['Procesado'] = 'No'

# Guardar el archivo CSV modificado
data.to_csv(file_path, index=False)

In [62]:
## import pandas as pd

# Realiza un reset de la columna Procesado, para volver a procesar
# Leer el archivo CSV
file_path = 'data/data_peliculas.csv'
data = pd.read_csv("data/data_peliculas.csv", low_memory=False)

# Cambia filas concretas a No para que se vuelvan a descargar
data.loc[3248:3250, 'Procesado'] = 'No'

# Guardar el archivo CSV modificado
data.to_csv(file_path, index=False)

In [ ]:
*****************************************************

In [46]:
import pandas as pd

# Borra todas las columnas menos la columna Procesado, para volver a procesar
# Cargar el archivo CSV en un DataFrame de pandas
file_path = 'data/data_peliculas.csv'
data = pd.read_csv(file_path, low_memory=False)

# Obtener una lista de las columnas a eliminar
columns_to_drop = [col for col in data.columns if col not in ['Title', 'URL', 'Procesado']]

# Eliminar las columnas no deseadas
data = data.drop(columns=columns_to_drop, axis=1)

# Guardar el DataFrame resultante en un nuevo archivo CSV
data.to_csv('data/data_peliculas.csv', index=False)

In [ ]:
*****************************************************

In [ ]:
HERRAMIENTAS DE AJUSTE DF*****************************

In [ ]:
*****************************************************

In [65]:
import pandas as pd

# Cargar el archivo CSV en un DataFrame
file_path = 'data/data_peliculas.csv'
df = pd.read_csv(file_path)

# 1. Eliminar ".0" de la columna "Votos"
df['Votos'] = df['Votos'].astype(str).str.replace('.0', '', regex=False)

# 2. Eliminar ".0" de la columna "Año"
df['Año'] = df['Año'].astype(str).str.replace('.0', '', regex=False)

# 3. Cambiar ";" por ", " en la columna "Coproduccion"
df['Coproduccion'] = df['Coproduccion'].str.replace(';', ', ', regex=False)

# Guardar el DataFrame corregido en un nuevo archivo CSV
df.to_csv('data/data_peliculas_corregido.csv', index=False)


In [66]:
# Corregir los nombres de las cabeceras de las columnas
df.columns = ['Título', 'URL', 'Procesado', 'Nota', 'Votos', 'Año', 'Reparto', 'Director', 'Compañías', 'Coproducción', 'Géneros', 'Premios']

# Guardar el DataFrame corregido en un nuevo archivo CSV
df.to_csv('data/data_peliculas_corregido.csv', index=False)


In [ ]:
*****************************************************

In [14]:
df

,Title,URL,Procesado,Nota,Votos,Año,Reparto,Director,Compañias,Coproduccion,Generos,Premios
0,El verdugo,https://www.filmaffinity.com/es/film411856.html,Sí,8.2,41447.0,1963.0,"José Isbert, Nino Manfredi, Emma Penella, José...",Luis García Berlanga,"Naga Films, Zabra Films",España-Italia;Naga,"Comedia, Sátira, Comedia negra, Película de culto",1963: Festival de Venecia: Premios FIPRESCI
1,Los santos inocentes,https://www.filmaffinity.com/es/film879812.html,Sí,8.1,44470.0,1984.0,"Alfredo Landa, Francisco Rabal, Juan Diego, Te...",Mario Camus,"Ganesh, RTVE",No se encontró información de coproducción,"Drama, Drama social, Vida rural, Años 60, Caza",1984: Cannes: Mejor Actor (ex aequo: Francisco...
2,Plácido,https://www.filmaffinity.com/es/film895672.html,Sí,8.1,20396.0,1961.0,"Cassen, José Luis López Vázquez, Elvira Quinti...",Luis García Berlanga,"Jet Films, Alfredo Matas",Films.Productor:Alfredo,"Comedia, Pobreza, Navidad, Sátira",1961: Nominada al Oscar: Mejor película de hab...
3,Viridiana,https://www.filmaffinity.com/es/film123112.html,Sí,8.1,28771.0,1961.0,"Silvia Pinal, Fernando Rey, Francisco Rabal, M...",Luis Buñuel,"Films 59, Uninci",España-México;Films,"Drama, Religión, Película de culto",1961: Festival de Cannes: Palma de Oro (ex-aequo)
4,"Bienvenido, Míster Marshall",https://www.filmaffinity.com/es/film435869.html,Sí,8.0,31421.0,1953.0,"José Isbert, Lolita Sevilla, Manolo Morán, Alb...",Luis García Berlanga,Uninci,No se encontró información de coproducción,"Comedia, Sátira",1953: Festival de Cannes: Premio Internacional...
...,...,...,...,...,...,...,...,...,...,...,...,...
3283,Mallorca Zombie,https://www.filmaffinity.com/es/film614286.html,Sí,1.6,236,2009,No se encontró información de reparto,Marc Albertí,"Alex Vargas, Friki Films","Vargas,Friki","Terror, Zombis, Serie B, Película de episodios",NaN
3284,El altar: Demasiado tarde para rezar,https://www.filmaffinity.com/es/film166981.html,Sí,1.6,123,2013,No se encontró información de reparto,Pedro Sánchez Román,Grafxdigital,No se encontró información de coproducción,"Terror, Fantástico",NaN
3285,Bocadillo,https://www.filmaffinity.com/es/film513377.html,Sí,1.5,2073,2018,"Ismael Prego, Joaquín Albero, David Cajal, Mar...",Ismael Prego,YouPlanet,No se encontró información de coproducción,Comedia,NaN
3286,2097,https://www.filmaffinity.com/es/film680362.html,Sí,1.5,102,2017,"Saray Castro, Abraham Santacruz, Alba Martín, ...",Esteban Calderín,No se encontró información de compañías,No se encontró información de coproducción,Acción,NaN


In [ ]:
************ HERRAMIENTAS DE VISUALIZACIÓN DE INFORMACIÓN SOBRE EL DATA ***********

In [ ]:
***********************************************************************************

In [17]:
import pandas as pd

# Cargar el archivo CSV en un DataFrame
file_path = 'data/data_peliculas_corregido.csv'
df_peliculas = pd.read_csv(file_path)

# Obtener una lista de todas las columnas en el DataFrame
column_names = df_peliculas.columns.tolist()
column_names

['Título',
 'URL',
 'Procesado',
 'Nota',
 'Votos',
 'Año',
 'Reparto',
 'Director',
 'Compañías',
 'Coproducción',
 'Géneros',
 'Premios']

In [18]:
import pandas as pd

# Cargar el archivo CSV en un DataFrame
file_path = 'data/data_peliculas_corregido.csv'
df_peliculas_corregido = pd.read_csv(file_path)

# Obtener un resumen estadístico del DataFrame
resumen = df_peliculas.describe(include='all').transpose()
print(resumen)

             count unique                                              top  \
Título        3288   3256                                     La distancia   
URL           3288   3288  https://www.filmaffinity.com/es/film411856.html   
Procesado     3288      1                                               Sí   
Nota          3288     68                                              5.5   
Votos         3288   1894                                              123   
Año           3288     93                                             2022   
Reparto       3288   3117            No se encontró información de reparto   
Director      3288   1172                                   Mariano Ozores   
Compañías     3266   2497          No se encontró información de compañías   
Coproducción  3288   1463                                            Films   
Géneros       3288   1825                                          Comedia   
Premios       1316   1232        2023: Festival de Málaga: Secci

In [23]:
import pandas as pd

# Cargar el archivo CSV en un DataFrame
file_path = 'data/data_peliculas_corregido.csv'
df_peliculas = pd.read_csv(file_path)

# Calcular la matriz de correlación para todas las columnas numéricas
correlation_matrix = df_peliculas.corr(numeric_only=True)

# Mostrar la matriz de correlación
print(correlation_matrix)


Empty DataFrame
Columns: []
Index: []


In [27]:
file_path = 'data/data_peliculas_corregido.csv'
df_peliculas = pd.read_csv(file_path)

# Mostrar el tipo de dato
print(df_peliculas.dtypes)

Título          object
URL             object
Procesado       object
Nota            object
Votos           object
Año             object
Reparto         object
Director        object
Compañías       object
Coproducción    object
Géneros         object
Premios         object
dtype: object


In [44]:
import pandas as pd

# Cargar el archivo CSV en un DataFrame
file_path = 'data/data_peliculas_corregido.csv'
df = pd.read_csv(file_path)

# Transformar los tipos de datos
df['Título'] = df['Título'].astype('string')
df['Nota'] = pd.to_numeric(df['Nota'], errors='coerce')
df['Votos'] = pd.to_numeric(df['Votos'], errors='coerce')
df['Año'] = pd.to_datetime(df['Año'], errors='coerce', format='%Y')


# Transformar columnas a listas de strings
def to_list_of_strings(cell):
    if pd.isna(cell):
        return []
    return [item.strip() for item in cell.split(',')]

columns_to_transform = ['Reparto', 'Director', 'Compañías', 'Coproducción', 'Géneros', 'Premios']
for col in columns_to_transform:
    df[col] = df[col].apply(to_list_of_strings)

# Guardar el DataFrame transformado en un nuevo archivo CSV
df.to_csv('data/data_peliculas_transformado.csv', index=False)


In [45]:
print(df.dtypes)

Título                  string
URL                     object
Procesado               object
Nota                   float64
Votos                  float64
Año             datetime64[ns]
Reparto                 object
Director                object
Compañías               object
Coproducción            object
Géneros                 object
Premios                 object
dtype: object


In [46]:
df

,Título,URL,Procesado,Nota,Votos,Año,Reparto,Director,Compañías,Coproducción,Géneros,Premios
0,El verdugo,https://www.filmaffinity.com/es/film411856.html,Sí,8.2,41447.0,1963-01-01,"[José Isbert, Nino Manfredi, Emma Penella, Jos...",[Luis García Berlanga],"[Naga Films, Zabra Films]","[España-Italia, Naga]","[Comedia, Sátira, Comedia negra, Película de c...",[1963: Festival de Venecia: Premios FIPRESCI]
1,Los santos inocentes,https://www.filmaffinity.com/es/film879812.html,Sí,8.1,44470.0,1984-01-01,"[Alfredo Landa, Francisco Rabal, Juan Diego, T...",[Mario Camus],"[Ganesh, RTVE]",[No se encontró información de coproducción],"[Drama, Drama social, Vida rural, Años 60, Caza]",[1984: Cannes: Mejor Actor (ex aequo: Francisc...
2,Plácido,https://www.filmaffinity.com/es/film895672.html,Sí,8.1,20396.0,1961-01-01,"[Cassen, José Luis López Vázquez, Elvira Quint...",[Luis García Berlanga],"[Jet Films, Alfredo Matas]",[Films.Productor:Alfredo],"[Comedia, Pobreza, Navidad, Sátira]",[1961: Nominada al Oscar: Mejor película de ha...
3,Viridiana,https://www.filmaffinity.com/es/film123112.html,Sí,8.1,28771.0,1961-01-01,"[Silvia Pinal, Fernando Rey, Francisco Rabal, ...",[Luis Buñuel],"[Films 59, Uninci]","[España-México, Films]","[Drama, Religión, Película de culto]",[1961: Festival de Cannes: Palma de Oro (ex-ae...
4,"Bienvenido, Míster Marshall",https://www.filmaffinity.com/es/film435869.html,Sí,8.0,31421.0,1953-01-01,"[José Isbert, Lolita Sevilla, Manolo Morán, Al...",[Luis García Berlanga],[Uninci],[No se encontró información de coproducción],"[Comedia, Sátira]",[1953: Festival de Cannes: Premio Internaciona...
...,...,...,...,...,...,...,...,...,...,...,...,...
3283,Mallorca Zombie,https://www.filmaffinity.com/es/film614286.html,Sí,1.6,236.0,2009-01-01,[No se encontró información de reparto],[Marc Albertí],"[Alex Vargas, Friki Films]","[Vargas, Friki]","[Terror, Zombis, Serie B, Película de episodios]",[]
3284,El altar: Demasiado tarde para rezar,https://www.filmaffinity.com/es/film166981.html,Sí,1.6,123.0,2013-01-01,[No se encontró información de reparto],[Pedro Sánchez Román],[Grafxdigital],[No se encontró información de coproducción],"[Terror, Fantástico]",[]
3285,Bocadillo,https://www.filmaffinity.com/es/film513377.html,Sí,1.5,2073.0,2018-01-01,"[Ismael Prego, Joaquín Albero, David Cajal, Ma...",[Ismael Prego],[YouPlanet],[No se encontró información de coproducción],[Comedia],[]
3286,2097,https://www.filmaffinity.com/es/film680362.html,Sí,1.5,102.0,2017-01-01,"[Saray Castro, Abraham Santacruz, Alba Martín,...",[Esteban Calderín],[No se encontró información de compañías],[No se encontró información de coproducción],[Acción],[]


In [ ]:
@@@@@@@@@@@@@@@@@@@ fin